In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE,ADASYN
from sklearn.compose import ColumnTransformer,make_column_selector as selector
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.under_sampling import RandomUnderSampler

In [15]:
url="https://raw.githubusercontent.com/digipodium/Datasets/main/classfication/bank_data.csv"
df=pd.read_csv(url,index_col="RowNumber")


In [16]:
df.head()
df.drop(columns=["CustomerId","Surname"],inplace=True)

In [18]:
x=df.drop(columns="Exited")
y=df.Exited

In [19]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")),
     ("scaler", StandardScaler())]
)


categorical_transformer = OneHotEncoder(handle_unknown="ignore")



In [20]:

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, x.select_dtypes(np.number).columns.to_list()),
        ("cat", categorical_transformer, x.select_dtypes("object").columns.to_list()),
    ]
)

In [21]:

clf=Pipeline(steps=[
    ("preprocessor",preprocessor),
("classifier",RandomForestClassifier(n_estimators=5))
])


In [22]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,random_state=0,test_size=.25)

In [23]:
param_grid={
    "classifier__min_samples_split":[2,4,5,6]
    
}
grid_search=GridSearchCV(estimator=clf,param_grid=param_grid,cv=3)

In [24]:
grid_search.fit(xtrain,ytrain)


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['CreditScore',
                                                                          'Age',
                                                                          'Tenure',
                                                                          'Balance',
                                                                          'NumOfProducts',
                                                                          'HasCrCard',
                                                                          'IsActiveMember',
                                                                          'EstimatedSalary']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Geography',
                                                                          'Gender'])])),
                                       ('classifier',
                                        RandomForestClassifier(n_estimators=5))]),
             param_grid={'classifier__min_samples_split': [2, 4, 5, 6]})

In [25]:
print("best param")
print(grid_search.best_params_)

best param
{'classifier__min_samples_split': 6}


In [26]:
clf.fit(xtrain,ytrain)
ypre=clf.predict(xtest)
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(ytest,ypre))
print(confusion_matrix(ytest,ypre))

              precision    recall  f1-score   support

           0       0.88      0.92      0.90      1991
           1       0.63      0.53      0.58       509

    accuracy                           0.84      2500
   macro avg       0.76      0.72      0.74      2500
weighted avg       0.83      0.84      0.84      2500

[[1834  157]
 [ 240  269]]


In [27]:
from joblib import dump
dump(clf,"churnmodel.jb")

['churnmodel.jb']